# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 18 2022 5:20PM,242161,21,402502728-1,"NBTY Global, Inc.",Released
1,Jul 18 2022 5:14PM,242160,21,402502712-1,"NBTY Global, Inc.",Released
2,Jul 18 2022 4:37PM,242159,20,8173919-FLS,"ACI Healthcare USA, Inc.",Released
3,Jul 18 2022 4:12PM,242158,16,8292185,Sartorius Lab Products and Service,Released
4,Jul 18 2022 4:12PM,242158,16,8292186,Sartorius Lab Products and Service,Released
5,Jul 18 2022 4:12PM,242158,16,8292191,Sartorius Lab Products and Service,Released
6,Jul 18 2022 4:12PM,242158,16,8292197,Sartorius Lab Products and Service,Released
7,Jul 18 2022 4:11PM,242157,16,8292173,Sartorius Bioprocess Solutions,Released
8,Jul 18 2022 3:52PM,242146,16,8290869,Sartorius Bioprocess Solutions,Released
9,Jul 18 2022 3:52PM,242146,16,8291446,Sartorius Bioprocess Solutions,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
43,242157,Released,1
44,242158,Released,4
45,242159,Released,1
46,242160,Released,1
47,242161,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242157,NaN,NaN,1.0
242158,NaN,NaN,4.0
242159,NaN,NaN,1.0
242160,NaN,NaN,1.0
242161,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241900,4.0,2.0,6.0
241972,2.0,8.0,4.0
242005,0.0,0.0,3.0
242007,0.0,0.0,1.0
242008,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241900,4,2,6
241972,2,8,4
242005,0,0,3
242007,0,0,1
242008,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241900,4,2,6
1,241972,2,8,4
2,242005,0,0,3
3,242007,0,0,1
4,242008,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241900,4,2,6
1,241972,2,8,4
2,242005,,,3
3,242007,,,1
4,242008,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 18 2022 5:20PM,242161,21,"NBTY Global, Inc."
1,Jul 18 2022 5:14PM,242160,21,"NBTY Global, Inc."
2,Jul 18 2022 4:37PM,242159,20,"ACI Healthcare USA, Inc."
3,Jul 18 2022 4:12PM,242158,16,Sartorius Lab Products and Service
7,Jul 18 2022 4:11PM,242157,16,Sartorius Bioprocess Solutions
8,Jul 18 2022 3:52PM,242146,16,Sartorius Bioprocess Solutions
10,Jul 18 2022 3:37PM,242147,21,"NBTY Global, Inc."
11,Jul 18 2022 3:22PM,242139,20,"ACI Healthcare USA, Inc."
79,Jul 18 2022 3:10PM,242096,10,Eywa Pharma Inc.
92,Jul 18 2022 3:05PM,242137,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 18 2022 5:20PM,242161,21,"NBTY Global, Inc.",,,1
1,Jul 18 2022 5:14PM,242160,21,"NBTY Global, Inc.",,,1
2,Jul 18 2022 4:37PM,242159,20,"ACI Healthcare USA, Inc.",,,1
3,Jul 18 2022 4:12PM,242158,16,Sartorius Lab Products and Service,,,4
4,Jul 18 2022 4:11PM,242157,16,Sartorius Bioprocess Solutions,,,1
5,Jul 18 2022 3:52PM,242146,16,Sartorius Bioprocess Solutions,,,2
6,Jul 18 2022 3:37PM,242147,21,"NBTY Global, Inc.",,,1
7,Jul 18 2022 3:22PM,242139,20,"ACI Healthcare USA, Inc.",,,68
8,Jul 18 2022 3:10PM,242096,10,Eywa Pharma Inc.,4,,9
9,Jul 18 2022 3:05PM,242137,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 18 2022 5:20PM,242161,21,"NBTY Global, Inc.",1,,
1,Jul 18 2022 5:14PM,242160,21,"NBTY Global, Inc.",1,,
2,Jul 18 2022 4:37PM,242159,20,"ACI Healthcare USA, Inc.",1,,
3,Jul 18 2022 4:12PM,242158,16,Sartorius Lab Products and Service,4,,
4,Jul 18 2022 4:11PM,242157,16,Sartorius Bioprocess Solutions,1,,
5,Jul 18 2022 3:52PM,242146,16,Sartorius Bioprocess Solutions,2,,
6,Jul 18 2022 3:37PM,242147,21,"NBTY Global, Inc.",1,,
7,Jul 18 2022 3:22PM,242139,20,"ACI Healthcare USA, Inc.",68,,
8,Jul 18 2022 3:10PM,242096,10,Eywa Pharma Inc.,9,,4
9,Jul 18 2022 3:05PM,242137,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 18 2022 5:20PM,242161,21,"NBTY Global, Inc.",1,,
1,Jul 18 2022 5:14PM,242160,21,"NBTY Global, Inc.",1,,
2,Jul 18 2022 4:37PM,242159,20,"ACI Healthcare USA, Inc.",1,,
3,Jul 18 2022 4:12PM,242158,16,Sartorius Lab Products and Service,4,,
4,Jul 18 2022 4:11PM,242157,16,Sartorius Bioprocess Solutions,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 18 2022 5:20PM,242161,21,"NBTY Global, Inc.",1,NaN,NaN
1,Jul 18 2022 5:14PM,242160,21,"NBTY Global, Inc.",1,NaN,NaN
2,Jul 18 2022 4:37PM,242159,20,"ACI Healthcare USA, Inc.",1,NaN,NaN
3,Jul 18 2022 4:12PM,242158,16,Sartorius Lab Products and Service,4,NaN,NaN
4,Jul 18 2022 4:11PM,242157,16,Sartorius Bioprocess Solutions,1,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 18 2022 5:20PM,242161,21,"NBTY Global, Inc.",1,0.0,0.0
1,Jul 18 2022 5:14PM,242160,21,"NBTY Global, Inc.",1,0.0,0.0
2,Jul 18 2022 4:37PM,242159,20,"ACI Healthcare USA, Inc.",1,0.0,0.0
3,Jul 18 2022 4:12PM,242158,16,Sartorius Lab Products and Service,4,0.0,0.0
4,Jul 18 2022 4:11PM,242157,16,Sartorius Bioprocess Solutions,1,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1694285,27,7.0,26.0
102,242008,1,0.0,0.0
16,726461,7,0.0,0.0
19,968514,11,0.0,15.0
20,1936441,143,18.0,8.0
21,3631740,15,0.0,0.0
22,484232,2,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1694285,27,7.0,26.0
1,102,242008,1,0.0,0.0
2,16,726461,7,0.0,0.0
3,19,968514,11,0.0,15.0
4,20,1936441,143,18.0,8.0
5,21,3631740,15,0.0,0.0
6,22,484232,2,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,27,7.0,26.0
1,102,1,0.0,0.0
2,16,7,0.0,0.0
3,19,11,0.0,15.0
4,20,143,18.0,8.0
5,21,15,0.0,0.0
6,22,2,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,27.0
1,102,Released,1.0
2,16,Released,7.0
3,19,Released,11.0
4,20,Released,143.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,16,19,20,21,22
Status,,,,,,,
Completed,26.0,0.0,0.0,15.0,8.0,0.0,0.0
Executing,7.0,0.0,0.0,0.0,18.0,0.0,0.0
Released,27.0,1.0,7.0,11.0,143.0,15.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,16,19,20,21,22
0,Completed,26.0,0.0,0.0,15.0,8.0,0.0,0.0
1,Executing,7.0,0.0,0.0,0.0,18.0,0.0,0.0
2,Released,27.0,1.0,7.0,11.0,143.0,15.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,16,19,20,21,22
0,Completed,26.0,0.0,0.0,15.0,8.0,0.0,0.0
1,Executing,7.0,0.0,0.0,0.0,18.0,0.0,0.0
2,Released,27.0,1.0,7.0,11.0,143.0,15.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()